In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

def renameColumns(df):
    return [col.split('-', 1)[1] if '-' in col else col for col in df.columns]

def convertTime(df):
    if df["Time"].dtype == 'int64':
        df["Time"] = pd.to_datetime(df["Time"], unit='ms')    
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')
    df.sort_values(by=['Time'], inplace=True)
    return df

In [3]:
def readAndProcessCSV(filename):
    df = pd.read_csv(filename)
    df.columns = renameColumns(df)
    df = convertTime(df)
    return df

In [4]:
def isWeekend(dateObj):
    return dateObj.weekday() in [5, 6]

def isWeeekday(dateObj):
    return not isWeekend(dateObj)

#Depending on on the mask created, you can get the data from the weekend or the week
def applyMask(df, mask):
    dfWeekend = df[~mask]
    dfWeekend = dfWeekend.reset_index(drop=True)
    return dfWeekend

def eliminateNightHours(df):
    return df[(df['Time'].dt.hour >= 6) & (df['Time'].dt.hour <= 23)]

def eliminateDayHours(df):
    return df[(df['Time'].dt.hour > 6) & (df['Time'].dt.hour < 23)]

In [5]:
def makeDfSensorSingle(df, sensorName):
    sensorColumns = ['Time'] + [col for col in df.columns if sensorName in col] 
    dfSensor = df[sensorColumns]
    dfSensor = dfSensor.dropna(axis=1, how='all')
    dfSensor = dfSensor.dropna().reset_index(drop=True)
    return dfSensor

In [6]:
#DF that has all the data from October and November
dfOctNov = readAndProcessCSV("sensorData/08_45_18.csv")
fileNames = ["08_46_27", "08_47_03", "08_48_14", "08_48_51", "08_49_32", "08_50_13", "08_50_46"]
for fileName in fileNames:
    df = readAndProcessCSV("sensorData/" + fileName + ".csv")
    dfOctNov = pd.concat([dfOctNov, df])

#Time range of dfOctNov is 2023-10-01 to 2023-11-25
#dfOctNov["Time"].dt.date

In [7]:
#DF that has only date time data (6 AM to 11 PM)
dfDay = eliminateNightHours(dfOctNov)
dfDay = dfDay.reset_index(drop=True)

In [8]:

#dfWeekend - df that contains only the data from the weekend
weekendMask = dfOctNov['Time'].apply(isWeekend)
dfWeekend = applyMask(dfOctNov, weekendMask)

In [9]:
#dfWeekday - df that contains only the data from the week
weekdayMask = dfOctNov['Time'].apply(isWeeekday)
dfWeekday = applyMask(dfOctNov, weekdayMask)

In [30]:
dfFinal = applyMask(dfOctNov, weekdayMask)
dfFinal = eliminateNightHours(dfFinal)
dfFinal = dfFinal.reset_index(drop=True)
#eye01 2023-10-01 06:09:37
#eye02 2023-10-01 06:07:36
#eye03 2023-10-01 06:06:06
#eye04 2023-10-01 06:04:44
#eye05 2023-10-01 06:09:40
#eye06 2023-10-01 06:06:08
#eye07 2023-10-01 06:07:59
#eye08 2023-10-01 06:09:25
#eye09 2023-10-01 06:06:47
#eye10 2023-10-01 06:03:35
#eye11 2023-10-01 06:05:22

#gas01 2023-10-01 06:05:59
#gas02 2023-10-01 06:06:37
#gas03 2023-10-01 06:05:37
#gas04 2023-10-01 06:05:29
#gas05 2023-10-01 06:03:23
#gas06 2023-10-01 06:06:59

#voc01 2023-10-01 06:00:22
#voc02 2023-10-01 06:02:00
#voc03 2023-10-01 06:00:25
#voc04 2023-10-01 06:04:26
#voc05 2023-10-01 06:03:39

sensorEye01 = makeDfSensorSingle(dfFinal, 'voc05')
#get all rows where the column motion is >1
sensorEye01 =  

,Time,voc05.humidity,voc05.light,voc05.motion,voc05.temperature,voc05.tvoc,voc05.vdd
0,2023-10-01 06:03:39,37.0,0.0,0.0,26.2,86.0,3654.0
1,2023-10-01 06:08:39,37.0,0.0,0.0,26.2,86.0,3654.0
2,2023-10-01 06:13:39,37.0,0.0,0.0,26.2,95.0,3654.0
3,2023-10-01 06:18:39,37.0,0.0,0.0,26.2,91.0,3654.0
4,2023-10-01 06:23:39,37.0,0.0,0.0,26.2,92.0,3654.0


In [11]:
#Sensor reading for eye01 - every 10 minutes 00:09:38
#Sensor reading for eye02 - every 10 minutes 00:07:37 
#Sensor reading for eye03 - every 10 minutes 00:06:08
#eye10, eye04
sensorEye01 = makeDfSensorSingle(dfDay, 'eye01')
sensorGas01 = makeDfSensorSingle(dfOctNov, 'gas01')
sensorVOC01 = makeDfSensorSingle(dfOctNov, 'voc01')

In [12]:
#sensorEye01 = sensorEye01[sensorEye01['eye01.occupancy'] == 1]
sensorEye01.head()

,Time,eye01.humidity,eye01.light,eye01.motion,eye01.occupancy,eye01.temperature,eye01.vdd
0,2023-10-01 06:09:37,40.0,0.0,0.0,0.0,23.2,3662.0
1,2023-10-01 06:19:37,40.0,0.0,0.0,0.0,23.3,3664.0
2,2023-10-01 06:29:37,40.0,1.0,0.0,0.0,23.3,3662.0
3,2023-10-01 06:39:37,40.0,1.0,0.0,0.0,23.2,3662.0
4,2023-10-01 06:49:37,40.0,1.0,0.0,0.0,23.2,3664.0


In [13]:
sensorVOC01.head()

,Time,voc01.humidity,voc01.light,voc01.motion,voc01.temperature,voc01.tvoc,voc01.vdd
0,2023-10-01 00:00:23,30.0,0.0,0.0,25.3,131.0,3648.0
1,2023-10-01 00:05:23,30.0,0.0,0.0,25.3,119.0,3648.0
2,2023-10-01 00:10:23,31.0,0.0,0.0,25.3,135.0,3648.0
3,2023-10-01 00:15:23,31.0,0.0,0.0,25.3,128.0,3648.0
4,2023-10-01 00:20:23,31.0,0.0,0.0,25.3,132.0,3648.0


In [14]:
sensorGas01.head()

,Time,gas01.co2,gas01.humidity,gas01.light,gas01.motion,gas01.temperature,gas01.vdd
0,2023-10-01 00:06:00,430.0,34.0,0.0,0.0,25.8,3667.0
1,2023-10-01 00:16:00,438.0,34.0,0.0,0.0,25.7,3667.0
2,2023-10-01 00:26:00,442.0,34.0,0.0,0.0,25.7,3667.0
3,2023-10-01 00:36:00,436.0,34.0,0.0,0.0,25.8,3664.0
4,2023-10-01 00:46:00,435.0,34.0,0.0,0.0,25.8,3667.0
